In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nlpUtils as utils


In [2]:
# read in saved dataframe containing chapter text
unpickled_df = pd.read_pickle("./sla_chapter_text.pkl")
unpickled_df

,chapter_text,book,chapter
0,prelud stormlight archiv kalak round rocki st...,1,1
1,love men frigid thing mountain stream three s...,1,2
2,kill bastard kill sun still hot die day week ...,1,3
3,ten order love forsaken us almighti shard sou...,1,4
4,man stood watch homeland fall dust water surg...,1,5
...,...,...,...
456,discard help potenti ﬁnal passion el ﬁrst fin...,4,127
457,ye look forward rule human el ﬁrst final ten ...,4,128
458,nearli much look forward serv newest odium re...,4,129
459,fourteen month ago eshonai hit ground chasm f...,4,130


In [3]:
# define constants
numTopics = 5 # number of topics or clusters
low_doc_freq = 0.01
high_doc_freq = 0.75
do_LDA = False

In [4]:
# vectorize chapter text
vectorized_matrix, vectorizer = utils.vectorizeTextIDF(unpickled_df['chapter_text'], low_doc_freq, high_doc_freq) # for NMF and kMeans
if do_LDA: vectorized_matrix_noidf, vectorizer_noidf = utils.vectorizeText(unpickled_df['chapter_text'], low_doc_freq, high_doc_freq) # for LDA

In [5]:
# do NMF
doc_top_matrix = utils.doNMF(numTopics, vectorized_matrix, vectorizer)

# do kMeans clustering
doc_clusters = utils.dokMeans(numTopics, vectorized_matrix, vectorizer)

# do LDA
if do_LDA: lda_doc_probs = utils.doLDA(numTopics, vectorized_matrix_noidf, vectorizer_noidf)


NMF Topic Words:
Topic 0: shallan jasnah veil pattern tyn balat
Topic 1: kaladin teft syl bridg moash rock
Topic 2: dalinar navani taravangian sadea vision gavilar
Topic 3: venli eshonai raboniel rhythm human rlain
Topic 4: adolin veil honorspren sadea duel spren

kMeans Cluster Words:
Cluster 0: dalinar adolin navani sadea parshendi vision
Cluster 1: kaladin syl teft bridg moash dalinar
Cluster 2: venli eshonai rhythm human raboniel attun
Cluster 3: navani szeth taravangian raboniel kal lirin
Cluster 4: shallan adolin jasnah veil pattern father


In [6]:
# update dataframe with document topics and clusters
df_with_results = unpickled_df.copy()
df_with_results['NMF'] = doc_top_matrix[:].tolist()
df_with_results['kMeans'] = doc_clusters
if do_LDA: df_with_results['LDA'] = lda_doc_probs
df_with_results

,chapter_text,book,chapter,NMF,kMeans
0,prelud stormlight archiv kalak round rocki st...,1,1,"[0.009663985483293154, 0.012663406625214776, 0...",3
1,love men frigid thing mountain stream three s...,1,2,"[0.01881734352792629, 0.034969191961122734, 0....",3
2,kill bastard kill sun still hot die day week ...,1,3,"[0.0, 0.1689605108405236, 0.015554179946130849...",1
3,ten order love forsaken us almighti shard sou...,1,4,"[0.021797438212706216, 0.22489481974900707, 0....",1
4,man stood watch homeland fall dust water surg...,1,5,"[0.19157066229959405, 0.012148789363697458, 0....",4
...,...,...,...,...,...
456,discard help potenti ﬁnal passion el ﬁrst fin...,4,127,"[0.0, 0.10458516676225763, 0.08681888147287979...",3
457,ye look forward rule human el ﬁrst final ten ...,4,128,"[0.10988394398969996, 0.0, 0.0, 0.235568225526...",2
458,nearli much look forward serv newest odium re...,4,129,"[0.0, 0.14942247459774322, 0.20916077685501033...",1
459,fourteen month ago eshonai hit ground chasm f...,4,130,"[0.009208951985547257, 0.02213448318691414, 0....",2
